# **NLP Query Interface for Spreadsheet Database Using Dexie.js and LLM**

This specification details the design and implementation of an NLP-driven interface that allows users to interact with a spreadsheet-like database using **natural language queries**. The system leverages **Dexie.js** for database interactions and an **LLM (Large Language Model)** for query interpretation and summarization.

---

## **1. System Overview**

The system comprises the following components:

1. **User Interface (Chat System)**: Accepts natural language queries from the user.
2. **NLP Layer (LLM Integration)**: Translates natural language queries into structured JSON queries.
3. **Database Interaction Layer (Dexie.js)**: Executes queries on the database.
4. **Post-Processing Layer**: Summarizes and processes results for user-friendly output.

---

## **2. Database Schema**

The database uses Dexie.js with the following tables:

### **Table: `results`**
| Field         | Type    | Description                        |
|---------------|---------|------------------------------------|
| `id`          | String  | Unique ID: combination of `row_column`. |
| `row`         | String  | Input row identifier.              |
| `column`      | String  | Prompt column identifier.          |
| `input`       | String  | Input text.                       |
| `prompt`      | String  | Prompt text.                      |
| `result`      | String  | Generated result from processing. |
| `version`     | Number  | Version number.                   |
| `updatedAt`   | Number  | Timestamp of last update.         |

### **Table: `result_history`**
| Field         | Type    | Description                        |
|---------------|---------|------------------------------------|
| `historyId`   | String  | Unique history ID (`row_column_v#`).|
| `resultId`    | String  | Reference to the result ID.        |
| `version`     | Number  | Version number.                   |
| `input`       | String  | Input text.                       |
| `prompt`      | String  | Prompt text.                      |
| `result`      | String  | Result at that version.           |
| `updatedAt`   | Number  | Timestamp of the version.         |

### **Table: `transaction_log`**
| Field         | Type    | Description                        |
|---------------|---------|------------------------------------|
| `transactionId` | String | Unique ID for the transaction.    |
| `operation`   | String  | INSERT, UPDATE, DELETE.           |
| `table`       | String  | Table name affected.              |
| `row`         | String  | Row ID.                           |
| `column`      | String  | Column ID.                        |
| `oldValue`    | Object  | Previous value.                   |
| `newValue`    | Object  | New value.                        |
| `timestamp`   | Number  | Time of operation.                |

---

In [ ]:
import Dexie from "dexie";

// Initialize the database
const db = new Dexie("SpreadsheetDB");
db.version(1).stores({
  results: "id, row, column, version, updatedAt",
  result_history: "historyId, resultId, version, updatedAt",
  transaction_log: "transactionId, timestamp, table, operation"
});

console.log("Database initialized.");

In [ ]:
async function sendToLLM(prompt, data = null) {
  const response = await fetch("https://api.openai.com/v1/chat/completions", {
    method: "POST",
    headers: {
      "Content-Type": "application/json",
      Authorization: `Bearer YOUR_API_KEY`
    },
    body: JSON.stringify({
      model: "gpt-4",
      messages: [
        { role: "system", content: prompt },
        { role: "user", content: data ? JSON.stringify(data) : "" }
      ],
      temperature: 0
    })
  });

  const result = await response.json();
  return result.choices[0].message.content;
}

In [ ]:
async function executeQuery(query) {
  const { action, table, conditions } = query;

  if (action === "query") {
    let dexieQuery = db[table];

    for (const field in conditions) {
      dexieQuery = dexieQuery.where(field).equals(conditions[field]);
    }

    return await dexieQuery.toArray();
  }

  if (action === "analyze_focus") {
    const allResults = await db.results.toArray();
    return allResults;
  }

  throw new Error("Unsupported query action.");
}

In [ ]:
async function processNaturalQuery(userQuery) {
  // Step 1: Ask LLM to parse the query
  const structuredQuery = await sendToLLM(
    "Convert the user's natural language query into a structured JSON for database queries.",
    userQuery
  );

  const parsedQuery = JSON.parse(structuredQuery);

  // Step 2: Execute the parsed query
  const results = await executeQuery(parsedQuery);

  // Step 3: Summarize the results using the LLM
  const summary = await sendToLLM(
    "Summarize the following query results for the user:",
    results
  );

  return summary;
}

In [ ]:
<div>
  <input id="userQuery" type="text" placeholder="Ask about the database..." />
  <button onclick="runStrategicQuery()">Submit</button>
  <div id="response"></div>
</div>
<script>
  async function runStrategicQuery() {
    const query = document.getElementById("userQuery").value;
    const response = await processNaturalQuery(query);
    document.getElementById("response").innerText = response;
  }
</script>